# Bond Option Comparison with DerivaGem Function 17

Value an option on a coupon paying bond using the Hull-White in response to a question.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree
from financepy.products.bonds import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  12 Jul 2021 at 11:46 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Set up the Discount Curve

In [3]:
settlement_date = Date(1, 12, 2019)

Set up discount curve

In [4]:
rate = 0.05
dcType = DayCountTypes.ACT_ACT_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settlement_date, rate, fixedFreq, dcType)

## Set up the Bond Option

First create the bond

In [5]:
issue_date = Date(1,12,2018)
maturity_date = issue_date.add_tenor("10Y")
coupon = 0.05
frequencyType = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_date, maturity_date, coupon, frequencyType, accrual_type)

Let's first price the bond on the libor curve

In [6]:
cp = bond.clean_price_from_discount_curve(settlement_date, discount_curve)
fp = bond.full_price_from_discount_curve(settlement_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   102.500


In [7]:
settlement_date

01-DEC-2019

In [8]:
bond.print_flows(settlement_date)

 01-DEC-2019          2.50 
 01-JUN-2020          2.50 
 01-DEC-2020          2.50 
 01-JUN-2021          2.50 
 01-DEC-2021          2.50 
 01-JUN-2022          2.50 
 01-DEC-2022          2.50 
 01-JUN-2023          2.50 
 01-DEC-2023          2.50 
 01-JUN-2024          2.50 
 01-DEC-2024          2.50 
 01-JUN-2025          2.50 
 01-DEC-2025          2.50 
 01-JUN-2026          2.50 
 01-DEC-2026          2.50 
 01-JUN-2027          2.50 
 01-DEC-2027          2.50 
 01-JUN-2028          2.50 
 01-DEC-2028        102.50 


Then define and create the option

In [9]:
expiry_date = settlement_date.add_tenor("18m")
strike_price = 100.0
face_amount = 100.0

In [10]:
europeanCallBondOption = BondOption(bond, expiry_date, strike_price, face_amount, FinOptionTypes.EUROPEAN_CALL)
americanCallBondOption = BondOption(bond, expiry_date, strike_price, face_amount, FinOptionTypes.AMERICAN_CALL)

In [11]:
europeanPutBondOption = BondOption(bond, expiry_date, strike_price, face_amount, FinOptionTypes.EUROPEAN_PUT)
americanPutBondOption = BondOption(bond, expiry_date, strike_price, face_amount, FinOptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [12]:
cp = bond.clean_price_from_discount_curve(expiry_date, discount_curve)
fp = bond.full_price_from_discount_curve(expiry_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Full  Price: %9.3f"% fp)

Fixed Income Clean Price:    99.993
Fixed Income Full  Price:   102.493


Set the model parameters, start with zero vol.

In [20]:
num_steps = 100

In [21]:
sigma = 0.0125
a = 0.1
modelHW = HWTree(sigma, a, num_steps)

In [22]:
ec = europeanCallBondOption.value(settlement_date, discount_curve, modelHW)
ac = americanCallBondOption.value(settlement_date, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   2.41836
American Call Value:   2.63792


In [23]:
ec = europeanPutBondOption.value(settlement_date, discount_curve, modelHW)
ac = americanPutBondOption.value(settlement_date, discount_curve, modelHW)
print("European Put Value: %9.5f" % ec)
print("American Put Value: %9.5f" % ac)

European Put Value:   2.44005
American Put Value:   2.78186


# COMPARISON

In [27]:
if 1==1:
    settlementDate = Date(1, 12, 2019)
    issueDate = Date(1, 12, 2018)
    expiryDate = settlementDate.add_tenor("18m")
    maturityDate = settlementDate.add_tenor("10Y")
    coupon = 0.05
    freqType = FrequencyTypes.SEMI_ANNUAL
    accrualType = DayCountTypes.ACT_ACT_ICMA
    bond = Bond(issueDate, maturityDate, coupon, freqType, accrualType)

    couponTimes = []
    couponFlows = []
    cpn = bond._coupon/bond._frequency

    numFlows = len(bond._flow_dates)
    for i in range(0, numFlows):

        pcd = bond._flow_dates[i-1]
        ncd = bond._flow_dates[i]

        if ncd > settlementDate:
            
            if len(couponTimes) == 0:
                flowTime = (pcd - settlementDate) / gDaysInYear
                couponTimes.append(flowTime)
                couponFlows.append(cpn)
                
            flowTime = (ncd - settlementDate) / gDaysInYear
            couponTimes.append(flowTime)
            couponFlows.append(cpn)

    couponTimes = np.array(couponTimes)
    couponFlows = np.array(couponFlows)

    strikePrice = 100.0
    face = 100.0
    y = 0.05
    times = np.linspace(0, 10, 21)
    dfs = np.power(1+y/2, -times*2)

    model = HWTree(sigma, a, None)

    #  Test convergence
    texp = (expiryDate - settlementDate)/gDaysInYear
    tmat = (maturityDate - settlementDate)/gDaysInYear

    # Jamshidian approach
    vJam = model.european_bond_option_jamshidian(texp, strikePrice, face,
                                              couponTimes, couponFlows,
                                              times, dfs)

    model._num_time_steps = 100
    model.build_tree(tmat, times, dfs)
    exerciseType = FinExerciseTypes.EUROPEAN
    vHW = model.bond_option(texp, strikePrice, face,
                            couponTimes, couponFlows, exerciseType)


In [28]:
vJam

{'call': 2.5571679355702432, 'put': 2.58259155047988}

In [29]:
vHW

{'call': 2.5627764182698214, 'put': 2.5835704651747804}

Copyright (c) Dominic O'Kane 2021